# SQL Alchemy CORE
This sintax allows for an easier query

In [37]:
from sqlalchemy.sql import select
from sqlalchemy import create_engine, inspect
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

In [38]:
engine = create_engine('sqlite:///test.db')

## Create Table and insert

In [39]:
conn = engine.connect()

In [40]:
metadata = MetaData()
users = Table('users', metadata, Column('id', Integer, primary_key=True), Column('name', String),Column('fullname', String))

In [41]:
addresses = Table('addresses', metadata, Column('id', Integer, primary_key=True), Column('user_id', None, ForeignKey('users.id')), 
                  Column('email_address', String, nullable=False))

In [42]:
metadata.create_all(engine)

In [43]:
ins = users.insert().values(name='jack', fullname='Jack Jones')
result = conn.execute(ins)

In [44]:
ins = users.insert()
conn.execute(ins, name='wendy', fullname='Wendy Williams')

In [45]:
conn.execute(addresses.insert(), [
{'user_id': 1, 'email_address' : 'jack@yahoo.com'},
{'user_id': 1, 'email_address' : 'jack@msn.com'},
{'user_id': 2, 'email_address' : 'www@www.org'},
{'user_id': 2, 'email_address' : 'wendy@aol.com'},
])

In [46]:
conn.execute(users.update().values(name='wendy', fullname='Wendy Jones').where(users.c.id== 2))

## Reflecting the Metadata of an existing DB Table
<https://docs.sqlalchemy.org/en/latest/core/reflection.html>

### Single Table

In [47]:
meta = MetaData()

In [48]:
users = Table('users', meta, autoload=True, autoload_with=engine)

In [49]:
[c.name for c in users.columns]

# pprint.pprint(meta)

['id', 'name', 'fullname']

### All Tables

In [52]:
meta = MetaData()
meta.reflect(bind=engine)

In [53]:
len(meta.sorted_tables)

2

Alternatively an inspector can be used

In [54]:
insp = inspect(engine)

In [55]:
insp.get_columns('users', schema=None)
insp.get_foreign_keys('users')
insp.get_schema_names()

['main']

In [56]:
print(insp.get_table_names())

['addresses', 'users']


In [57]:
insp.get_table_names()
users_columns  =insp.get_columns('users', schema=None)
users_columns[0]

{'name': 'id',
 'type': INTEGER(),
 'nullable': False,
 'default': None,
 'autoincrement': 'auto',
 'primary_key': 1}

## SELECT 

In [58]:
s = select([users])
result = conn.execute(s)

In [59]:
s = select([users.c.name, users.c.fullname])

In [60]:
conn.execute(s).fetchone()

('jack', 'Jack Jones')

In [61]:
from sqlalchemy.sql import and_, or_, not_

In [62]:
# Select name, fullname from users where id <2 and name = 'jack'
s = select([users.c.name, users.c.fullname]).where(and_(users.c.id <2, users.c.name =='jack'))
conn.execute(s).fetchall()

[('jack', 'Jack Jones')]

## Joins

In [63]:
s = select([users, addresses]).where(users.c.id == addresses.c.user_id)
conn.execute(s).fetchall()

[(1, 'jack', 'Jack Jones', 1, 1, 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 2, 1, 'jack@msn.com'),
 (2, 'wendy', 'Wendy Jones', 3, 2, 'www@www.org'),
 (2, 'wendy', 'Wendy Jones', 4, 2, 'wendy@aol.com'),
 (1, 'jack', 'Jack Jones', 5, 1, 'jack@yahoo.com'),
 (1, 'jack', 'Jack Jones', 6, 1, 'jack@msn.com'),
 (2, 'wendy', 'Wendy Jones', 7, 2, 'www@www.org'),
 (2, 'wendy', 'Wendy Jones', 8, 2, 'wendy@aol.com')]